In [1]:
from src.service.HwpService import HwpService

service = HwpService()
filePath = '/Users/dobby/Desktop/Workspace/3월 계측보고서(김해상동물류)_간단이.hwp'

xml = service.hwp2xml(filePath)
columnTag, textTag = service.findTag(xml, '일자별 계측 현황')
xmlData = service.setTableData(columnTag, textTag)

In [2]:
tableId = sorted(set([i['table-id'] for i in xmlData]))
xmlData = [[i for i in xmlData if i['table-id'] == id] for id in tableId]


In [3]:
targetText = [ '구분', '진동속도', '진동레벨', '소음', '허용기준', '비고' ]

filteredData = [
    sublist for sublist in xmlData
    if any( entry['row'] in [1, 2] and any(keyword in entry['text'] for keyword in targetText) for entry in sublist )
]

filteredData


[[{'table-id': 90,
   'row': 0,
   'rowspan': 1,
   'col': 0,
   'colspan': 1,
   'text': '3월4일'},
  {'table-id': 90, 'row': 0, 'rowspan': 1, 'col': 1, 'colspan': 1, 'text': ''},
  {'table-id': 90, 'row': 0, 'rowspan': 1, 'col': 2, 'colspan': 1, 'text': ''},
  {'table-id': 90, 'row': 0, 'rowspan': 1, 'col': 3, 'colspan': 1, 'text': ''},
  {'table-id': 90, 'row': 0, 'rowspan': 1, 'col': 4, 'colspan': 1, 'text': ''},
  {'table-id': 90, 'row': 0, 'rowspan': 1, 'col': 5, 'colspan': 1, 'text': ''},
  {'table-id': 90, 'row': 0, 'rowspan': 1, 'col': 6, 'colspan': 1, 'text': ''},
  {'table-id': 90, 'row': 0, 'rowspan': 1, 'col': 7, 'colspan': 1, 'text': ''},
  {'table-id': 90, 'row': 0, 'rowspan': 1, 'col': 8, 'colspan': 1, 'text': ''},
  {'table-id': 90,
   'row': 1,
   'rowspan': 2,
   'col': 0,
   'colspan': 1,
   'text': '구분'},
  {'table-id': 90,
   'row': 1,
   'rowspan': 1,
   'col': 1,
   'colspan': 2,
   'text': '진동속도(cm/s)'},
  {'table-id': 90,
   'row': 1,
   'rowspan': 1,
   'col': 

In [32]:
import copy
from collections import defaultdict, Counter
from itertools import cycle, chain
from src.dto.CustomDefaultDict import CustomDefaultDict
        
def expandData(data: dict):
    expandData = []

    for i in range(data['colspan']):
        tempData = copy.deepcopy(data)
        tempData['colspan'] = 1
        tempData['col'] += i

        expandData.append(tempData)

    return expandData

def createRange(dateDataList):
    rowCounts = Counter(data['row'] for data in dateDataList)
    dateRow = sorted(list({row: count for row, count in rowCounts.items() if count == 1}.keys()))
    
    ranges = defaultdict(range)
    if len(dateRow) < 2:
         ranges[0] = range(dateRow[0] + 1, max(rowCounts.keys()) + 1)
    else:
        for index in range(len(dateRow)):
            startIndex = dateRow[index - 1] + 1
            endIndex = dateRow[index] - 1
            if startIndex <= endIndex:
                ranges[dateRow[index - 1]] = range(startIndex, endIndex + 1)
        
        ranges[dateRow[-1]] = range(endIndex + 2, max(rowCounts.keys()) + 1)

    return dateRow, dict(ranges)

def test2(dictData: list[dict]):
    dateRowList, dateRanges = createRange(dictData)

    dataList = list(chain.from_iterable(expandData(data) for data in dictData if data['row'] not in dateRowList + [1, 2]))
    columns = list(chain.from_iterable(expandData(column) for column in dictData if column['row'] in [1]))

    for dateRow in dateRowList:
        for row in dateRanges[dateRow]:
            if row not in [dateRow + 1, dateRow + 2]:
                customDict = CustomDefaultDict()
                customDict.append('일시', next(date['text'] for date in dictData if date['row'] in [dateRow]))
                rowsGroupList = [data for data in dataList if data['row'] in [row]]
                for df, column in zip(rowsGroupList, columns):
                    customDict.append(column['text'], df['text'])
                print(dict(customDict))


            # r = CustomDefaultDict()
            #     # print(f'{column['text']}: {df['text']}')
            #     r.append(column['text'], df['text'])
            # print(r)
    

test2([dictData for dictData in filteredData[0] if dictData['text']])

    


{'일시': '3월4일', '구분': '대감터널', '진동속도(cm/s)': ['N/T', 'N/T'], '진동레벨[dB(V)]': ['N/T', 'N/T'], '소음[dB(A)]': ['N/T', 'N/T'], '허용기준': '1.0cm/s', '비고': 'O.K'}
{'일시': '3월4일', '구분': '애니시스템', '진동속도(cm/s)': ['0.021', '0.027'], '진동레벨[dB(V)]': ['47.55', '53.42'], '소음[dB(A)]': ['18.24', '84.55'], '허용기준': '0.2cm/s 75dB(V)75dB(A)', '비고': 'O.K'}
{'일시': '3월6일', '구분': '대감터널', '진동속도(cm/s)': ['0.125', '0.375'], '진동레벨[dB(V)]': ['-', '-'], '소음[dB(A)]': ['-', '-'], '허용기준': '1.0cm/s', '비고': 'O.K'}
{'일시': '3월6일', '구분': 'GS물류센터', '진동속도(cm/s)': ['0.039', '0.057'], '진동레벨[dB(V)]': ['54.6', '58.83'], '소음[dB(A)]': ['57.96', '63.62'], '허용기준': '0.2cm/s 75dB(V)75dB(A) ', '비고': 'O.K'}
{'일시': '3월7일', '구분': '대감터널', '진동속도(cm/s)': ['0.146', '0.24'], '진동레벨[dB(V)]': ['-', '-'], '소음[dB(A)]': ['-', '-'], '허용기준': '1.0cm/s', '비고': 'O.K'}
{'일시': '3월7일', '구분': 'GS물류센터', '진동속도(cm/s)': ['0.022', '0.054'], '진동레벨[dB(V)]': ['52.69', '62.13'], '소음[dB(A)]': ['24.21', '60.66'], '허용기준': '0.2cm/s 75dB(V)75dB(A)', '비고': 'O.K'}
{'일시': '3월9일', '구

In [10]:
A = [ '진동', '소음레벨' ]

'진동' in A

True